In [1]:
import numpy as np

In [2]:
class Operation:
    """Represents a Node in the Computation Graph"""
    
    def __init__(self, input_nodes = []):
        """Constructs an Operation with input_nodes as inputs
           which computes outputs to zero or more consumers"""
        self.input_nodes = input_nodes
        self.consumers = []
        self.inputs = [] # evaluated after each forward pass, TODO: np.array?
        self.output = None # evaluated after each forward pass
        
        # Connect this node with its inputs, by adding it as a consumer to its inputs
        for input_node in self.input_nodes:
            input_node.consumers.append(self)
        
        # Add this operation to the Computation Graph
        # TODO: provide the graph explicitly
        _default_graph.operations.append(self)
    
    def compute(self):
        """Computes the output of the operation. Depends on the specific operation."""
        pass

In [3]:
class Add(Operation):
    def __init__(self, x, y):
        super().__init__(input_nodes=[x, y])
    
    def compute(self, x, y):
        return x + y
    
    def gradient(self, accum_gradient):
        a = self.inputs[0]
        b = self.inputs[1]

        # TODO: can't really understand why we do all this summation
        grad_wrt_a = accum_gradient        
        while np.ndim(grad_wrt_a) > len(a.shape):
            grad_wrt_a = np.sum(grad_wrt_a, axis=0)
        for axis, size in enumerate(a.shape):
            if size == 1:
                grad_wrt_a = np.sum(grad_wrt_a, axis=axis, keepdims=True)

        grad_wrt_b = accum_gradient
        while np.ndim(grad_wrt_b) > len(b.shape):
            grad_wrt_b = np.sum(grad_wrt_b, axis=0)
        for axis, size in enumerate(b.shape):
            if size == 1:
                grad_wrt_b = np.sum(grad_wrt_b, axis=axis, keepdims=True)

        return [grad_wrt_a, grad_wrt_b]

In [4]:
class Matmul(Operation):
    def __init__(self, A, B):
        super().__init__(input_nodes=[A, B])

    def compute(self, A, B):
        return A.dot(B)
    
    def gradient(self, accum_gradient):
        A = self.inputs[0]
        B = self.inputs[1]
        # Because of gradients like this we cannot simply
        # apply the chain rule by multiplying by the accumulated gradient.
        # For the transpose matrices below, see Goodfellow's Deep Learning book, page 216
        return [accum_gradient.dot(B.T), A.T.dot(accum_gradient)]

In [5]:
class Sigmoid(Operation):
    def __init__(self, x):
        super().__init__(input_nodes=[x])
    
    def compute(self, x):
        return 1 / (1 + np.exp(-x))
    
    def gradient(self, accum_gradient):
        return self.output * (1 - self.output) * accum_gradient

In [6]:
class Softmax(Operation):
    def __init__(self, x):
        super().__init__(input_nodes=[x])
    
    def compute(self, x):
        """The input of Softmax is a vector"""
        # using vector operations
        # axis=1 so that for each row we sum its colums
        # the sum will eat 1 dimension, so we broadcast with [:, None]
        return np.exp(x) / np.sum(np.exp(x), axis=1)[:, None]
    
    def gradient(self, accum_gradient):
        softmax = self.output
        
        # TODO: can't really understand why this is correct
        return (accum_gradient - np.reshape(
            np.sum(accum_gradient * softmax, 1),
            [-1, 1]
        )) * softmax

In [7]:
# softmax playground
softmax = np.array([0.3, 0.2, 0.4, 0.1])

upstream_grad = np.array([0.1, 0.1, 0.1, 0.1])
# np.sum(upstream_grad * softmax.T, 1)
np.reshape([1, 2], [-1, 1])

array([[1],
       [2]])

In [8]:
class Log(Operation):
    def __init__(self, x):
        super().__init__(input_nodes=[x])
    
    def compute(self, x):
        return np.log(x)
    
    def gradient(self, accum_gradient):
        x = self.inputs[0]
        return (1 / x) * accum_gradient

In [70]:
class Multiply(Operation):
    """Element-wise multiplication of 2 matrices A and B"""
    
    def __init__(self, A, B):
        super().__init__(input_nodes=[A, B])

    def compute(self, A, B):
        return A * B
    
    def gradient(self, accum_gradient):
        A = self.inputs[0]
        B = self.inputs[1]
        
        # dimensionality of the gradient must match the inputs
        # derivative of multiplication wrt A is B and wrt to B is A
        return np.array([B, A]) * accum_gradient

In [34]:
class ReduceSum(Operation):
    """Computes the sum of the given tensor A based on the given axis.
       axis=None computes the sum of the whole tensor A.
    """
    
    def __init__(self, A, axis=None):
        super().__init__(input_nodes=[A])
        self.axis = axis

    def compute(self, A):
        return np.sum(A, axis=self.axis)
    
    def gradient(self, accum_gradient):
        A = self.inputs[0]
        
        output_shape = np.array(A.shape)

        # mark the dimension which was squashed by the sum in the forward pass
        output_shape[self.axis] = 1
        
        # set all dimensions to 1, except the summing dimenion(s)
        tile_scaling = A.shape // output_shape
        
        # restore the dimension(s), destroyed by the sum from the forward pass
        upstream_grad = np.reshape(accum_gradient, output_shape)
        
        # copy the upstream gradient over the dimension(s) which was squashed by the sum
        # so that it matches the dimensions of the given input
        return np.tile(upstream_grad, tile_scaling)

In [11]:
# playground

A = np.array([[1, 2, 3], [3, 4, 5]])
axis = 1
output = np.sum(A, axis)
grad = output / 100

output_shape = np.array(A.shape)
output_shape[axis] = 1
tile_scaling = A.shape // output_shape
# np.tile(A, tile_scaling)
upstream_grad = np.reshape(grad, output_shape)

np.tile(upstream_grad, tile_scaling)

array([[0.06, 0.06, 0.06],
       [0.12, 0.12, 0.12]])

In [12]:
class Negate(Operation):
    def __init__(self, x):
        super().__init__(input_nodes=[x])

    def compute(self, x):
        return -x
    
    def gradient(self, accum_gradient):
        return -1 * accum_gradient

In [13]:
class Placeholder:
    """Represents an input node which doesn't have any inputs
       and can only be consumed by other Nodes in the Computation Graph.
       
       The Placeholder has a fixed value. Acts like a constant."""
    
    def __init__(self):
        self.consumers = []
        
        # Register the placeholder in the Computation Graph
        # TODO: provide the graph explicitly
        _default_graph.placeholders.append(self)

In [14]:
class Variable:
    """Represents a parameter in the Computation Graph.
       This node doesn't have any inputs and has only consumers.
       
       The Variable's value can change. It is initialized to initial_value."""
    
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.consumers = []
        
        # Register the variable in the Computation Graph
        # TODO: provide the graph explicitly
        _default_graph.variables.append(self)

In [15]:
class Graph:
    """Represents the actual Computation Graph which has 3 types of Nodes:
       - placeholders
       - variables
       - operations
    """
    
    def __init__(self, placeholders=[], variables=[], operations=[]):
        self.placeholders = placeholders
        self.variables = variables
        self.operations = operations

    def as_default(self):
        global _default_graph
        _default_graph = self
        return _default_graph

In [16]:
class Session:
    """Represents a single execution of the whole Computation graph."""
    # TODO: provide the Graph explicitly
    
    def run(self, operation, feed_dict={}):
        """Performs a post-order traversal of all nodes in the Computation graph,
           so that all operations with known inputs are performed first.
        """
        
        # TODO: cache this order
        nodes_in_post_order = Session.traverse_post_order(operation)
        
        # TODO: use some kind of environment to store evaluated inputs and outputs of nodes
        outputs = {operation: None for operation in nodes_in_post_order}
        
        for node in nodes_in_post_order:
            if type(node) == Placeholder:
                outputs[node] = feed_dict[node]
            elif type(node) == Variable:
                outputs[node] = node.value
            elif isinstance(node, Operation):
                computed_inputs = [outputs[input_node] for input_node in node.input_nodes]
                node.inputs = computed_inputs
                
                outputs[node] = node.compute(*computed_inputs)
                node.output = outputs[node]
                
        return outputs[operation]

    @staticmethod
    def traverse_post_order(operation):
        operations_post_order = []
        
        def traverse(node):
            # Placeholders and Variables do not have input_nodes
            if isinstance(node, Operation):
                for input_node in node.input_nodes:
                    traverse(input_node)

            operations_post_order.append(node)
        
        traverse(operation)
        return operations_post_order

In [17]:
# Linear Perceptron

graph = Graph().as_default()

A = Variable(np.array([
    [1, 0],
    [0, -1]
]))
b = Variable(np.array([1, 1]))

x = Placeholder()

y = Add(Matmul(A, x), b)

Session().run(y, feed_dict={
    x: np.array([1, 2])
})

array([ 2, -1])

In [18]:
# Sigmoid Perceptron

graph = Graph().as_default()

x = Placeholder()
w = Variable(initial_value=np.random.normal(0, 1, 2))
b = Variable(initial_value=np.random.normal(0, 1))

perceptron = Sigmoid(Add(Matmul(w, x), b))

Session().run(perceptron, feed_dict={
    x: np.array([-1, 1])
})

0.038057472201464576

In [19]:
# Multi-class Perceptron

graph = Graph().as_default()

training_examples = np.array([
    [-3, -3],
    [-3, -4],
    [4, 5],
    [3, 6]
])

labels = np.array([
    [0, 1],
    [0, 1],
    [1, 0],
    [1, 0]
])

# will be a matrix used for batch computation
X = Placeholder()

W = Variable(np.array([
    [1, -1],
    [1, -1]
]))

b = Variable(np.array([0, 0]))

classifier = Softmax(Add(Matmul(X, W), b))

Session().run(classifier, {
    X: training_examples
})

array([[6.14417460e-06, 9.99993856e-01],
       [8.31528028e-07, 9.99999168e-01],
       [9.99999985e-01, 1.52299795e-08],
       [9.99999985e-01, 1.52299795e-08]])

In [20]:
# Cross-entropy loss

C = Placeholder()

# TODO: not sure about the ReduceSum over all dimensions here
cross_entropy_loss = Negate(ReduceSum(
    Multiply(C, Log(classifier))
))

Session().run(cross_entropy_loss, {
    X: training_examples,
    C: labels
})

7.006181810554592e-06

In [67]:
# Gradient Descent Optimizer

class GradientDescentOptimizer:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
    
    def minimize(self, loss):
        learning_rate = self.learning_rate

        class Minimize(Operation):
            def __init__(self):
                super().__init__(input_nodes=[loss])
            
            # TODO: this loss argument to compute is unneeded, because we just want to hook into the graph
            def compute(self, loss):
                # TODO: how to inject compute_gradients or just the gradients at each step?
                loss_node = self.input_nodes[0]
                gradients_table = compute_gradients(loss_node)

#                 print('gradients_table: {}'.format(gradients_table))
                
                for node, gradient in gradients_table.items():
                    if type(node) == Variable:
                        node.value -= learning_rate * gradient
        
        return Minimize()

In [68]:
# Backwards pass of Backpropagation which computes the gradients

from queue import Queue

def compute_gradients(loss):
    gradients_table = {}
    
    # the loss gradient with regard to itself is 1
    gradients_table[loss] = 1
    
    visited = set()
    queue = Queue()
    queue.put(loss)
    visited.add(loss)
    
    # BFS from loss backwards to inputs
    while not queue.empty():
        node = queue.get()

#         if hasattr(node, 'consumers'):
        if node != loss: # the loss doesn't have any consumers
            # compute gradient of node

            # stores gradient of the loss wrt the node's output
            gradients_table[node] = 0
            
#             print('node: {}'.format(node))
#             print('node.input_nodes: {}'.format(node.input_nodes))
            
            for consumer in node.consumers:
                # get the accumulated gradient for the consumer
                loss_gradient_wrt_consumer_output = gradients_table[consumer]
                
                # TODO: use multiplication for the chain rule and do not pass gradients around
                # actually it is not so trivial to do that
                
#                 print('consumer: {}'.format(consumer))
                
                # apply the chain rule, using consumer's accumulated gradient, to compute...
                # TODO: consumer.gradient or node.gradient?
                loss_gradient_wrt_consumer_inputs = consumer.gradient(loss_gradient_wrt_consumer_output)
                
#                 print('loss_gradient_wrt_consumer_inputs: {}'.format(loss_gradient_wrt_consumer_inputs))
                
                if len(consumer.input_nodes) == 1:
#                     print('trying to add {} to gradient {}'.format(loss_gradient_wrt_consumer_inputs, gradients_table[node]))
                    # the consumer has only 1 input (the current node), thus consumer's input is scalar
                    gradients_table[node] += loss_gradient_wrt_consumer_inputs
                else:
                    # otherwise the consumer consumes a vector
                    # find the index of node in consumer's inputs
                    node_index_in_consumer_inputs = consumer.input_nodes.index(node)
                    
                    # get the gradient only from the edge from node to consumer
                    loss_gradient_wrt_node = loss_gradient_wrt_consumer_inputs[node_index_in_consumer_inputs]
                    
                    # accumulate the gradient for that consumer
                    gradients_table[node] += loss_gradient_wrt_node

        # continue backwards to node's inputs
        if hasattr(node, 'input_nodes'):
            for input_node in node.input_nodes:
                # Placeholder nodes do not have a gradient, so ignore them
                if not input_node in visited and not type(input_node) == Placeholder:
                    queue.put(input_node)
                    visited.add(input_node)
    
    return gradients_table

In [79]:
# Multi-class Perceptron

np.random.seed(42)

graph = Graph().as_default()

training_examples = np.array([
    [-3, -3],
    [-3, -4],
    [4, 5],
    [3, 6]
])

labels = np.array([
    [0, 1],
    [0, 1],
    [1, 0],
    [1, 0]
])

X = Placeholder()
C = Placeholder()

# W = Variable(np.array([
#     [1, -1],
#     [1, -1]
# ]))
# b = Variable(np.array([0, 0]))

# Initialize weights randomly
W = Variable(np.random.randn(2, 2))
b = Variable(np.random.randn(2))


perceptron_classifier = Softmax(Add(Matmul(X, W), b))

# TODO: not sure about the ReduceSum over all dimensions here
cross_entropy_loss = Negate(ReduceSum(
    Multiply(C, Log(perceptron_classifier))
))

# Build minimization operation
minimization_op = GradientDescentOptimizer(learning_rate=0.01).minimize(cross_entropy_loss)

# Build placeholder inputs
feed_dict = {
    X: training_examples,
    C: labels
}

# Create session
session = Session()

# Perform 100 gradient descent steps
for step in range(100):
    loss_value = session.run(cross_entropy_loss, feed_dict)
    if step % 10 == 0:
        print("Step:", step, " Loss:", loss_value)
    session.run(minimization_op, feed_dict)

# Print final result
W_value = session.run(W)
print("Weight matrix:\n", W_value)
b_value = session.run(b)
print("Bias:\n", b_value)

Step: 0  Loss: 8.26458938989594
Step: 10  Loss: 0.1652251421349391
Step: 20  Loss: 0.08367745749183048
Step: 30  Loss: 0.05668636296948762
Step: 40  Loss: 0.04309110803322651
Step: 50  Loss: 0.03486166620694781
Step: 60  Loss: 0.029328832181125745
Step: 70  Loss: 0.025346073543178964
Step: 80  Loss: 0.02233789199912719
Step: 90  Loss: 0.019983150161384144
Weight matrix:
 [[ 0.87961879 -0.52116893]
 [ 1.20170703  0.96901137]]
Bias:
 [-0.23746006 -0.23083027]
